In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import copy

import numpy as np # linear algebra
from numpy.random import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import networkx as nx
print 'networkx: ', nx.__version__

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

datapath = '../data/hateful-users-on-twitter/'
# Any results you write to the current directory are saved as output.

networkx:  2.0


In [3]:
users = pd.read_csv(datapath + 'users_neighborhood_anon.csv',encoding='iso-8859-1')

In [4]:
users.head()

,user_id,hate,hate_neigh,normal_neigh,statuses_count,followers_count,followees_count,favorites_count,listed_count,betweenness,...,c_feminine_empath,c_medieval_empath,c_journalism_empath,c_farming_empath,c_plant_empath,c_shopping_empath,c_ship_empath,c_religion_empath,c_tourism_empath,c_power_empath
0,0,normal,True,True,101767,3504,3673,81635,53,100467.895084,...,0.001380,0.003288,0.000255,0.002189,0.000593,0.003689,0.003559,0.001634,0.002662,0.007487
1,1,other,False,False,2352,19609,309,61,197,0.000000,...,0.000802,0.004465,0.000444,0.001632,0.001298,0.002183,0.008969,0.004975,0.000647,0.003419
2,2,other,False,False,1044,2371,2246,561,16,4897.117853,...,0.000985,0.005284,0.000554,0.001084,0.001359,0.002041,0.016846,0.004881,0.001214,0.003347
3,3,other,False,False,167172,3004,298,3242,53,9.864754,...,0.001391,0.002061,0.001116,0.001282,0.000000,0.001951,0.015423,0.000446,0.000446,0.005241
4,4,other,False,False,1998,17643,19355,485,239,0.000000,...,0.000633,0.035382,0.000317,0.000475,0.000475,0.002431,0.007656,0.033273,0.072792,0.003698


In [5]:
edges = pd.read_csv(datapath + 'users.edges', header=None, delim_whitespace=True)
edges.head()

,0,1
0,10999,55317
1,10999,44622
2,10999,10999
3,10999,71821
4,10999,57907


In [6]:
G = nx.from_pandas_edgelist(edges, source=0, target=1)
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 100386
Number of edges: 2194979
Average degree:  43.7308


In [7]:
def random_seed(g):
    """this function recturns a single node from g, it's chosen with uniform probability"""
    ux = randint(0, g.number_of_nodes(), 1)
    return ux[0]

def snowball_sampling(g, seed, maxsize=50):
    """this function returns a set of nodes equal to maxsize from g that are 
    collected from around seed node via snownball sampling"""
    if g.number_of_nodes() < maxsize:
        return set()
    q = [seed]
    subgraph = set([seed])
    while q:
        top = q[0]
        q.remove(top)
        for node in g.neighbors(top):
            if len(subgraph) < maxsize:
                q.append(node)
                subgraph.add(node)
            else :
                return subgraph
    return subgraph

In [9]:
rs = random_seed(G)
nodes = snowball_sampling(G, rs, maxsize=2000)
GS = G.subgraph(nodes)
print(nx.info(GS))
print 'denisty:', nx.density(GS)
print 'components:', nx.number_connected_components(GS)
print 'clustering coeff:', nx.average_clustering(GS)

Name: 
Type: SubGraph
Number of nodes: 2000
Number of edges: 63834
Average degree:  63.8340
denisty: 0.0319329664832
components: 1
clustering coeff: 0.199732281796


In [139]:
gs_edges = nx.to_pandas_edgelist(GS)
gs_edges.head()

,source,target
0,14,57344
1,14,90116
2,14,90632
3,14,81930
4,14,4619


In [140]:
gs_node_features = users[users.user_id.isin(list(nodes))]
gs_nodes = gs_node_features[['user_id', 'hate']].copy()
hate_dict = {'hateful' : 1, 'normal' : -1, 'other': 0}
gs_nodes['hate'] = gs_nodes['hate'].apply(lambda x: hate_dict[x])
gs_nodes.head()

,user_id,hate
14,14,0
27,27,0
103,103,0
123,123,0
129,129,0


In [162]:
gs_features = gs_node_features.drop('hate', axis=1)
gs_features = gs_features.drop('hashtags', axis=1)
gs_features['hate_neigh'] = gs_features.hate_neigh.astype(int)
gs_features['normal_neigh'] = gs_features.normal_neigh.astype(int)
gs_features.head()

,user_id,hate_neigh,normal_neigh,statuses_count,followers_count,followees_count,favorites_count,listed_count,betweenness,eigenvector,...,c_feminine_empath,c_medieval_empath,c_journalism_empath,c_farming_empath,c_plant_empath,c_shopping_empath,c_ship_empath,c_religion_empath,c_tourism_empath,c_power_empath
14,14,1,1,109833,7629,6343,104280,189,71429.018382,4.102006e-06,...,0.000346,0.004871,0.000766,0.005759,0.000747,0.002922,0.006978,0.001354,0.001741,0.005541
27,27,0,0,2158,86,357,894,3,3454.744637,4.219037e-08,...,0.000633,0.004284,0.000790,0.001904,0.001269,0.003806,0.005376,0.001898,0.001585,0.006803
103,103,0,0,91713,19010,17432,609,251,0.000000,4.929853e-32,...,0.000836,0.007919,0.000729,0.009528,0.001124,0.002769,0.005859,0.001358,0.002222,0.005282
123,123,0,1,97883,13342,11451,154000,15,8753.643487,5.601360e-06,...,0.000141,0.004670,0.000383,0.004493,0.000975,0.003201,0.009501,0.002102,0.002129,0.004352
129,129,0,1,184976,5647,6167,66389,244,2265.116752,3.100713e-06,...,0.000607,0.005341,0.000451,0.005270,0.000451,0.002262,0.006846,0.001561,0.002011,0.003835


#### Fill the NaNs

In [163]:
gs_features = gs_features.fillna(0.0)
gs_features.isnull().sum().sum()

0L

#### Normalize all colums except 'user_id'

In [164]:
gsdf = gs_features.iloc[:, 1:]
normalized_gsf=(gsdf-gsdf.min())/(gsdf.max()-gsdf.min())
gs_features.iloc[:, 1:] = normalized_gsf
gs_features.head()

,user_id,hate_neigh,normal_neigh,statuses_count,followers_count,followees_count,favorites_count,listed_count,betweenness,eigenvector,...,c_feminine_empath,c_medieval_empath,c_journalism_empath,c_farming_empath,c_plant_empath,c_shopping_empath,c_ship_empath,c_religion_empath,c_tourism_empath,c_power_empath
14,14,1.0,1.0,0.183834,0.000185,0.024997,0.274502,0.002116,0.010546,0.004468,...,0.071132,0.192419,0.093558,0.466314,0.165252,0.187736,0.252121,0.087305,0.120047,0.228844
27,27,0.0,0.0,0.003551,0.000002,0.001407,0.002353,0.000034,0.000510,0.000046,...,0.130087,0.169223,0.096463,0.154189,0.280586,0.244500,0.194224,0.122373,0.109295,0.280988
103,103,0.0,0.0,0.153496,0.000461,0.068698,0.001603,0.002810,0.000000,0.000000,...,0.171792,0.312816,0.089118,0.771559,0.248520,0.177909,0.211686,0.087558,0.153199,0.218151
123,123,0.0,1.0,0.163826,0.000323,0.045128,0.405383,0.000168,0.001292,0.006101,...,0.028913,0.184491,0.046736,0.363794,0.215662,0.205648,0.343275,0.135530,0.146797,0.179745
129,129,0.0,1.0,0.309648,0.000137,0.024304,0.174759,0.002732,0.000334,0.003377,...,0.124655,0.210995,0.055119,0.426723,0.099682,0.145314,0.247363,0.100608,0.138686,0.158411


#### Write to csv files

In [165]:
gs_nodes.to_csv('../data/hateful/nodes.csv', header=None, index=None)
gs_edges.to_csv('../data/hateful/edges.csv', header=None, index=None)
gs_features.to_csv('../data/hateful/features.csv', header=None, index=None)